**Chapter 14 – Natural Language Processing with RNNs and Attention**

_This notebook contains all the sample code and solutions to the exercises in chapter 14._

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-mlp/blob/main/13_nlp_with_rnns_and_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-mlp/blob/main/13_nlp_with_rnns_and_attention.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Setup

This project requires Python 3.10 or above:

In [1]:
import sys

assert sys.version_info >= (3, 10)

And PyTorch ≥ 2.4.0:

In [2]:
from packaging.version import Version
import torch

assert Version(torch.__version__) >= Version("2.4.0")

If using Colab, a few libraries are not pre-installed so we must install them manually:

In [3]:
if "google.colab" in sys.modules:
    %pip install -q torchmetrics datasets

As we did in earlier chapters, let's define the default font sizes to make the figures prettier:

In [4]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

This chapter can be very slow without a hardware accelerator, so if we can find one, let's use it:

In [5]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

device

'mps'

Let's issue a warning if there's no hardware accelerator available:

In [6]:
# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if device == "cpu":
    print("Neural nets can be very slow without a hardware accelerator.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

And let's create the `images/nlp` folder (if it doesn't already exist), and define the `save_fig()` function which is used through this notebook to save the figures in high-res for the book:

In [7]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "nlp"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Generating Shakespearean Text Using a Character RNN

## Creating the Training Dataset

Let's download the Shakespeare data from Andrej Karpathy's [char-rnn project](https://github.com/karpathy/char-rnn/)

In [8]:
from pathlib import Path
import urllib.request

def download_shakespeare_text():
    path = Path("datasets/shakespeare/shakespeare.txt")
    if not path.is_file():
        path.parent.mkdir(parents=True, exist_ok=True)
        url = "https://homl.info/shakespeare"
        urllib.request.urlretrieve(url, path)
    return path.read_text()

shakespeare_text = download_shakespeare_text()

In [9]:
# extra code – shows a short text sample
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [10]:
vocab = sorted(set(shakespeare_text.lower()))
"".join(vocab)

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [11]:
char_to_id = {char: index for index, char in enumerate(vocab)}
id_to_char = {index: char for index, char in enumerate(vocab)}

In [12]:
char_to_id["a"]

13

In [13]:
id_to_char[13]

'a'

In [14]:
import torch

def encode_text(text):
    return torch.tensor([char_to_id[char] for char in text.lower()])

def decode_text(char_ids):
    return "".join([id_to_char[char_id.item()] for char_id in char_ids])

In [15]:
encoded = encode_text("Hello world!")
encoded

tensor([20, 17, 24, 24, 27,  1, 35, 27, 30, 24, 16,  2])

In [16]:
decode_text(encoded)

'hello world!'

In [17]:
from torch.utils.data import Dataset, DataLoader

class CharDataset(Dataset):
    def __init__(self, text, window_length):
        self.encoded_text = encode_text(text)
        self.window_length = window_length

    def __len__(self):
        return len(self.encoded_text) - self.window_length
    
    def __getitem__(self, idx):
        if idx >= len(self):
            raise IndexError("dataset index out of range")
        end = idx + self.window_length
        window = self.encoded_text[idx : end]
        target = self.encoded_text[idx + 1 : end + 1]
        return window, target

In [18]:
# extra code – a simple example using CharDataset
to_be_dataset = CharDataset("To be or not to be", window_length=10)
for x, y in to_be_dataset:
    print(f"x={x}, y={y}")
    print(f"    decoded: x={decode_text(x)!r}, y={decode_text(y)!r}")

x=tensor([32, 27,  1, 14, 17,  1, 27, 30,  1, 26]), y=tensor([27,  1, 14, 17,  1, 27, 30,  1, 26, 27])
    decoded: x='to be or n', y='o be or no'
x=tensor([27,  1, 14, 17,  1, 27, 30,  1, 26, 27]), y=tensor([ 1, 14, 17,  1, 27, 30,  1, 26, 27, 32])
    decoded: x='o be or no', y=' be or not'
x=tensor([ 1, 14, 17,  1, 27, 30,  1, 26, 27, 32]), y=tensor([14, 17,  1, 27, 30,  1, 26, 27, 32,  1])
    decoded: x=' be or not', y='be or not '
x=tensor([14, 17,  1, 27, 30,  1, 26, 27, 32,  1]), y=tensor([17,  1, 27, 30,  1, 26, 27, 32,  1, 32])
    decoded: x='be or not ', y='e or not t'
x=tensor([17,  1, 27, 30,  1, 26, 27, 32,  1, 32]), y=tensor([ 1, 27, 30,  1, 26, 27, 32,  1, 32, 27])
    decoded: x='e or not t', y=' or not to'
x=tensor([ 1, 27, 30,  1, 26, 27, 32,  1, 32, 27]), y=tensor([27, 30,  1, 26, 27, 32,  1, 32, 27,  1])
    decoded: x=' or not to', y='or not to '
x=tensor([27, 30,  1, 26, 27, 32,  1, 32, 27,  1]), y=tensor([30,  1, 26, 27, 32,  1, 32, 27,  1, 14])
    decoded: x=

In [19]:
window_length = 50
batch_size = 512  # reduce if your GPU cannot handle such a large batch size
train_set = CharDataset(shakespeare_text[:1_000_000], window_length)
valid_set = CharDataset(shakespeare_text[1_000_000:1_060_000], window_length)
test_set = CharDataset(shakespeare_text[1_060_000:], window_length)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)

## Embeddings

In [20]:
import torch.nn as nn

torch.manual_seed(42)
embed = nn.Embedding(5, 3)  # 5 categories × 3D embeddings
embed(torch.tensor([[3, 2], [0, 2]]))

tensor([[[ 0.2674,  0.5349,  0.8094],
         [ 2.2082, -0.6380,  0.4617]],

        [[ 0.3367,  0.1288,  0.2345],
         [ 2.2082, -0.6380,  0.4617]]], grad_fn=<EmbeddingBackward0>)

## Building and Training the Char-RNN Model

In [21]:
class ShakespeareModel(nn.Module):
    def __init__(self, vocab_size, n_layers=2, embedding_size=10, n_hidden=128,
                 dropout=0.1):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size)
        self.gru = nn.GRU(embedding_size, n_hidden, num_layers=n_layers,
                          batch_first=True, dropout=dropout)
        self.output = nn.Linear(n_hidden, vocab_size)

    def forward(self, X):
        embeddings = self.embed(X)
        outputs, _states = self.gru(embeddings)
        return self.output(outputs).permute(0, 2, 1)

In [22]:
import torchmetrics

def evaluate_tm(model, data_loader, metric):
    model.eval()
    metric.reset()
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            metric.update(y_pred, y_batch)
    return metric.compute()

def train(model, optimizer, loss_fn, metric, train_loader, valid_loader,
          n_epochs, patience=2, factor=0.5, epoch_callback=None):
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="max", patience=patience, factor=factor)
    history = {"train_losses": [], "train_metrics": [], "valid_metrics": []}
    for epoch in range(n_epochs):
        total_loss = 0.0
        metric.reset()
        model.train()
        if epoch_callback is not None:
            epoch_callback(model, epoch)
        for index, (X_batch, y_batch) in enumerate(train_loader):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            metric.update(y_pred, y_batch)
            train_metric = metric.compute().item()
            print(f"\rBatch {index + 1}/{len(train_loader)}", end="")
            print(f", loss={total_loss/(index+1):.4f}", end="")
            print(f", {train_metric=:.2%}", end="")
        history["train_losses"].append(total_loss / len(train_loader))
        history["train_metrics"].append(train_metric)
        val_metric = evaluate_tm(model, valid_loader, metric).item()
        history["valid_metrics"].append(val_metric)
        scheduler.step(val_metric)
        print(f"\rEpoch {epoch + 1}/{n_epochs},                      "
              f"train loss: {history['train_losses'][-1]:.4f}, "
              f"train metric: {history['train_metrics'][-1]:.2%}, "
              f"valid metric: {history['valid_metrics'][-1]:.2%}")
    return history

**Warning**: the following code may take a while to run, especially without a GPU.

In [23]:
torch.manual_seed(42)

model = ShakespeareModel(len(vocab)).to(device)

n_epochs = 20
xentropy = nn.CrossEntropyLoss()
optimizer = torch.optim.NAdam(model.parameters())
accuracy = torchmetrics.Accuracy(task="multiclass",
                                 num_classes=len(vocab)).to(device)

history = train(model, optimizer, xentropy, accuracy, train_loader, valid_loader,
                n_epochs)

Epoch 1/20,                      train loss: 1.6035, train metric: 51.30%, valid metric: 51.84%
Epoch 2/20,                      train loss: 1.3851, train metric: 56.71%, valid metric: 52.83%
Epoch 3/20,                      train loss: 1.3555, train metric: 57.45%, valid metric: 53.67%
Epoch 4/20,                      train loss: 1.3413, train metric: 57.80%, valid metric: 53.22%
Epoch 5/20,                      train loss: 1.3328, train metric: 58.02%, valid metric: 53.80%
Epoch 6/20,                      train loss: 1.3272, train metric: 58.15%, valid metric: 54.06%
Epoch 7/20,                      train loss: 1.3232, train metric: 58.25%, valid metric: 53.66%
Epoch 8/20,                      train loss: 1.3201, train metric: 58.33%, valid metric: 54.26%
Epoch 9/20,                      train loss: 1.3176, train metric: 58.39%, valid metric: 54.45%
Epoch 10/20,                      train loss: 1.3156, train metric: 58.44%, valid metric: 54.16%
Epoch 11/20,                      train

In [24]:
torch.save(model.state_dict(), "my_shakespeare_model.pt")

In [25]:
model.eval()  # don't forget to switch the model to evaluation mode!
text = "To be or not to b"
encoded_text = encode_text(text).unsqueeze(dim=0).to(device)
with torch.no_grad():
    Y_logits = model(encoded_text)
    predicted_char_id = Y_logits[0, :, -1].argmax().item()
    predicted_char = id_to_char[predicted_char_id]  # correctly predicts "e"

In [26]:
predicted_char

'e'

## Generating Shakespearean Text

In [27]:
torch.manual_seed(42)
probs = torch.tensor([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
samples = torch.multinomial(probs, replacement=True, num_samples=8)
samples

tensor([[0, 0, 0, 0, 1, 0, 2, 2]])

In [28]:
import torch.nn.functional as F

def next_char(model, text, temperature=1):
    encoded_text = encode_text(text).unsqueeze(dim=0).to(device)
    with torch.no_grad():
        Y_logits = model(encoded_text)
        Y_probas = F.softmax(Y_logits[0, :, -1] / temperature, dim=-1)
        predicted_char_id = torch.multinomial(Y_probas, num_samples=1).item()
    return id_to_char[predicted_char_id]

In [29]:
def extend_text(model, text, n_chars=80, temperature=1):
    for _ in range(n_chars):
        text += next_char(model, text, temperature)
    return text

In [30]:
print(extend_text(model, "To be or not to b", temperature=0.01))

To be or not to be the state
and the contrary of the state and the sea,
the common people of the 


In [31]:
print(extend_text(model, "To be or not to b", temperature=0.4))

To be or not to be the better from the cause
that thou think you may be so be gone.

romeo:
that 


In [32]:
print(extend_text(model, "To be or not to b", temperature=100))

To be or not to b-c3;m-rkn&:x:uyve:b&hi n;n-h;wt3k
&cixxh:a!kq$c$ 3 ncq$ ;;wq cp:!xq;yh
!3
d!nhi.


## Training a stateful RNN

Until now, we have only used _stateless RNNs_: at each training iteration the model starts with a hidden state full of zeros, then it updates this state at each time step, and after the last time step, it throws the state away as it is not needed anymore. What if we instructed the RNN to preserve this final state after processing a training batch and use it as the initial state for the next training batch? This way the model could learn long-term patterns despite only backpropagating through short sequences. This is called a _stateful RNN_. Let's go over how to build one.

The model itself requires very little change: we only need to add a new `hidden_states` attribute, initialized to `None`, then save the hidden states after each batch is processed, and use them as the initial hidden states for the next batch. Note that we must call `detach()` on these states to ensure we don't backpropagate over this training iteration's computation graph at the next iteration (this would cause an error).

In [33]:
class StatefulShakespeareModel(nn.Module):
    def __init__(self, vocab_size, n_layers=2, embedding_size=10, n_hidden=128,
                 dropout=0.1):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size)
        self.gru = nn.GRU(embedding_size, n_hidden, num_layers=n_layers,
                          batch_first=True, dropout=dropout)
        self.output = nn.Linear(n_hidden, vocab_size)
        self.hidden_states = None

    def forward(self, X):
        embeddings = self.embed(X)
        outputs, hidden_states = self.gru(embeddings, self.hidden_states)
        self.hidden_states = hidden_states.detach()
        return self.output(outputs).permute(0, 2, 1)

The main difficulty with stateful RNNs is preparing the dataset. Indeed, a stateful RNN only makes sense if each input sequence in a batch starts exactly where the corresponding sequence in the previous batch left off. To be more precise, the _n_<sup>th</sup> window in batch _k_ must start exactly where the _n_<sup>th</sup> window in batch _k_ – 1 stopped. For example, suppose the full encoded text is `[1, 2, 3, .., 59, 60, 61]` and you want to use a window length of 4, and a batch size of 5. The dataset could contain 3 batches like these, in this order:

```
Batch #1:
X=[[1,2,3,4], [13,14,15,16], [25,26,27,28], [37,38,39,40], [49,50,51,52]]
Y=[[2,3,4,5], [14,15,16,17], [26,27,28,29], [38,39,40,41], [50,51,52,53]]

Batch #2:
X=[[5,6,7,8], [17,18,19,20], [29,30,31,32], [41,42,43,44], [53,54,55,56]]
y=[[6,7,8,9], [18,19,20,21], [30,31,32,33], [42,43,44,45], [54,55,56,57]]

Batch #3:
X=[[9,10,11,12], [21,22,23,24], [33,34,35,36], [45,46,47,48], [57,58,59,60]]
y=[[10,11,12,13], [22,23,24,25], [34,35,36,37], [46,47,48,49], [58,59,60,61]]
```

Let's write a `StatefulCharDataset` class that organizes the data like this.

In [34]:
from torch.utils.data import Dataset, DataLoader

class StatefulCharDataset(Dataset):
    def __init__(self, text, window_length, batch_size):
        self.encoded_text = encode_text(text)
        self.window_length = window_length
        self.batch_size = batch_size
        n_consecutive_windows = (len(self.encoded_text) - 1) // window_length
        n_windows_per_slot = n_consecutive_windows // batch_size
        self.length = n_windows_per_slot * batch_size
        self.spacing = n_windows_per_slot * window_length

    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        if idx >= len(self):
            raise IndexError("dataset index out of range")
        start = ((idx % self.batch_size) * self.spacing
                 +(idx // self.batch_size) * self.window_length)
        end = start + self.window_length
        window = self.encoded_text[start : end]
        target = self.encoded_text[start + 1 : end + 1]
        return window, target

Now let's create the data loaders. Note that we must *not* shuffle the batches, even for the training set. We must also ensure that all batches have exactly the same number of windows, even the very last batch: for this reason, we must set `drop_last=True` when creating the data loaders.

In [35]:
batch_size = 128
stateful_train_set = StatefulCharDataset(shakespeare_text[:1_000_000],
                                         window_length, batch_size)
stateful_train_loader = DataLoader(stateful_train_set, batch_size=batch_size,
                                   drop_last=True)
stateful_valid_set = StatefulCharDataset(shakespeare_text[1_000_000:1_060_000],
                                         window_length, batch_size)
stateful_valid_loader = DataLoader(stateful_valid_set, batch_size=batch_size,
                                   drop_last=True)
stateful_test_set = StatefulCharDataset(shakespeare_text[1_060_000:],
                                        window_length, batch_size)
stateful_test_loader = DataLoader(stateful_test_set, batch_size=batch_size,
                                  drop_last=True)

During training, we should reset the hidden states at the start of each epoch. We could rewrite the whole training loop just for that, but it's cleaner to just pass a callback function to the `train()` function:

**Warning**: the following cell may take a long time to run, especially without a GPU.

In [36]:
torch.manual_seed(42)

stateful_model = StatefulShakespeareModel(len(vocab)).to(device)

n_epochs = 10
xentropy = nn.CrossEntropyLoss()
optimizer = torch.optim.NAdam(stateful_model.parameters())
accuracy = torchmetrics.Accuracy(task="multiclass",
                                 num_classes=len(vocab)).to(device)

def reset_hidden_states(model, epoch):
    model.hidden_states = None

history = train(stateful_model, optimizer, xentropy, accuracy, stateful_train_loader,
                stateful_valid_loader, n_epochs, epoch_callback=reset_hidden_states)

Epoch 1/10,                      train loss: 2.4736, train metric: 29.84%, valid metric: 39.19%
Epoch 2/10,                      train loss: 1.8788, train metric: 44.47%, valid metric: 45.18%
Epoch 3/10,                      train loss: 1.6925, train metric: 49.30%, valid metric: 48.28%
Epoch 4/10,                      train loss: 1.5978, train metric: 51.71%, valid metric: 49.84%
Epoch 5/10,                      train loss: 1.5416, train metric: 53.16%, valid metric: 51.01%
Epoch 6/10,                      train loss: 1.5034, train metric: 54.10%, valid metric: 51.79%
Epoch 7/10,                      train loss: 1.4751, train metric: 54.83%, valid metric: 52.46%
Epoch 8/10,                      train loss: 1.4538, train metric: 55.33%, valid metric: 52.89%
Epoch 9/10,                      train loss: 1.4360, train metric: 55.81%, valid metric: 53.17%
Epoch 10/10,                      train loss: 1.4222, train metric: 56.11%, valid metric: 53.81%


In [37]:
torch.save(stateful_model.state_dict(), "my_stateful_shakespeare_model.pt")

Let's try generating some text with our stateful RNN. We must reset the `hidden_states` every time we call the model:

In [38]:
def extend_text_with_stateful_rnn(model, text, n_chars=80, temperature=1):
    for _ in range(n_chars):
        model.hidden_states = None
        text += next_char(model, text, temperature)
    return text + "…"

In [39]:
torch.manual_seed(42)
stateful_model.eval()
print(extend_text_with_stateful_rnn(stateful_model, "To be or not to b",
                                    temperature=0.1))

To be or not to be
that the senate the senate the senate the seather.

clarence:
i shall be the d…


In [40]:
print(extend_text(stateful_model, "To be or not to b", temperature=0.4))

To be or not to be
the body and the commanding to the duke
that he shall be so did not side,
wher…


In [41]:
print(extend_text(stateful_model, "To be or not to b", temperature=1))

To be or not to be.

mortam:
will abless must; that chuses, as cack noble;
with a life that hath.…


# Sentiment Analysis

## Loading the IMDB Dataset

In [42]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
split = imdb_dataset["train"].train_test_split(train_size=0.8, seed=42)
imdb_train_set, imdb_valid_set = split["train"], split["test"]
imdb_test_set = imdb_dataset["test"]

In [43]:
imdb_train_set[1]["text"]

"'The Rookie' was a wonderful movie about the second chances life holds for us and also puts an emotional thought over the audience, making them realize that your dreams can come true. If you loved 'Remember the Titans', 'The Rookie' is the movie for you!! It's the feel good movie of the year and it is the perfect movie for all ages. 'The Rookie' hits a major home run!"

In [44]:
imdb_train_set[1]["label"]

1

In [45]:
imdb_train_set[16]["text"]

"Lillian Hellman's play, adapted by Dashiell Hammett with help from Hellman, becomes a curious project to come out of gritty Warner Bros. Paul Lukas, reprising his Broadway role and winning the Best Actor Oscar, plays an anti-Nazi German underground leader fighting the Fascists, dragging his American wife and three children all over Europe before finding refuge in the States (via the Mexico border). They settle in Washington with the wife's wealthy mother and brother, though a boarder residing in the manor is immediately suspicious of the newcomers and spends an awful lot of time down at the German Embassy playing poker. It seems to take forever for this drama to find its focus, and when we realize what the heart of the material is (the wise, honest, direct refugees teaching the clueless, head-in-the-sand Americans how the world has suddenly changed), it seems a little patronizing--the viewer is quite literally put in the relatives' place, being lectured to. Lukas has several speeches 

In [46]:
imdb_train_set[16]["label"]

0

## Tokenization Using the `tokenizers` Library

### Training a BPE Tokenizer on the IMDB Dataset

In [47]:
import tokenizers

bpe_model = tokenizers.models.BPE(unk_token="<unk>")
bpe_tokenizer = tokenizers.Tokenizer(bpe_model)
bpe_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Whitespace()
special_tokens = ["<pad>", "<unk>"]
bpe_trainer = tokenizers.trainers.BpeTrainer(vocab_size=1000,
                                             special_tokens=special_tokens)
train_reviews = [review["text"].lower() for review in imdb_train_set]
bpe_tokenizer.train_from_iterator(train_reviews, bpe_trainer)

In [48]:
tokenizers.pre_tokenizers.Whitespace().pre_tokenize_str("Hello, world!!!")

[('Hello', (0, 5)), (',', (5, 6)), ('world', (7, 12)), ('!!!', (12, 15))]

### Encoding and Decoding Text

In [49]:
some_review = "what an awesome movie! 😊"
bpe_encoding = bpe_tokenizer.encode(some_review)
bpe_encoding

Encoding(num_tokens=8, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [50]:
bpe_encoding.tokens

['what', 'an', 'aw', 'es', 'ome', 'movie', '!', '<unk>']

In [51]:
bpe_token_ids = bpe_encoding.ids
bpe_token_ids

[303, 139, 373, 149, 240, 211, 4, 1]

In [52]:
bpe_tokenizer.get_vocab()["what"]

303

In [53]:
bpe_tokenizer.token_to_id("what")

303

In [54]:
bpe_tokenizer.id_to_token(305)

'ough'

In [55]:
bpe_tokenizer.decode(bpe_token_ids)

'what an aw es ome movie !'

In [56]:
bpe_encoding.offsets

[(0, 4), (5, 7), (8, 10), (10, 12), (12, 15), (16, 21), (21, 22), (23, 24)]

### Handling Batches

In [57]:
bpe_tokenizer.encode_batch(train_reviews[:3])

[Encoding(num_tokens=281, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=114, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=285, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [58]:
bpe_tokenizer.enable_padding(pad_id=0, pad_token="<pad>")
bpe_tokenizer.enable_truncation(max_length=500)

In [59]:
bpe_encodings = bpe_tokenizer.encode_batch(train_reviews[:3])
bpe_batch_ids = torch.tensor([encoding.ids for encoding in bpe_encodings])
bpe_batch_ids

tensor([[159, 402, 176, 246,  61, [...], 215, 156, 586,   0,   0,   0,   0],
        [ 10, 138, 198, 289, 175, [...],   0,   0,   0,   0,   0,   0,   0],
        [289,  15, 209, 398, 177, [...],  50,  29,  22,  17,  24,  18,  24]])


In [60]:
attention_mask = torch.tensor([encoding.attention_mask
                               for encoding in bpe_encodings])
attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, [...], 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, [...], 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, [...], 1, 1, 1, 1, 1, 1, 1]])


In [61]:
lengths = attention_mask.sum(dim=-1)
lengths

tensor([281, 114, 285])

### BBPE Tokenization

The 😊 emoji is represented as 4 bytes when using the UTF-8 Unicode encoding, so the `ByteLevel` pre-tokenizer represents it as 4 characters, each representing a byte. Spaces are converted to Ġ.

In [62]:
tokenizers.pre_tokenizers.ByteLevel().pre_tokenize_str(some_review)

[('Ġwhat', (0, 4)),
 ('Ġan', (4, 7)),
 ('Ġawesome', (7, 15)),
 ('Ġmovie', (15, 21)),
 ('!', (21, 22)),
 ('ĠðŁĺĬ', (22, 24))]

In [63]:
bbpe_model = tokenizers.models.BPE(unk_token="<unk>")
bbpe_tokenizer = tokenizers.Tokenizer(bbpe_model)
bbpe_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.ByteLevel()
bbpe_trainer = tokenizers.trainers.BpeTrainer(vocab_size=1000,
                                              special_tokens=special_tokens)
bbpe_tokenizer.train_from_iterator(train_reviews, bbpe_trainer)

In [64]:
bbpe_encoding = bbpe_tokenizer.encode(some_review)
bbpe_tokens = bbpe_encoding.tokens
bbpe_tokens

['Ġwhat',
 'Ġan',
 'Ġaw',
 'es',
 'ome',
 'Ġmovie',
 '!',
 'Ġ',
 '<unk>',
 'Ł',
 'ĺ',
 '<unk>']

In [65]:
bbpe_token_ids = bbpe_encoding.ids
bbpe_token_ids

[354, 216, 561, 148, 244, 232, 2, 107, 1, 125, 119, 1]

In [66]:
bbpe_decoded = bbpe_tokenizer.decode(bbpe_token_ids)
bbpe_decoded

'Ġwhat Ġan Ġaw es ome Ġmovie ! Ġ Ł ĺ'

In [67]:
bbpe_decoded.replace(" ", "").replace("Ġ", " ").strip()

'what an awesome movie! Łĺ'

### WordPiece

In [68]:
wp_model = tokenizers.models.WordPiece(unk_token="<unk>")
wp_tokenizer = tokenizers.Tokenizer(wp_model)
wp_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Whitespace()
wp_trainer = tokenizers.trainers.WordPieceTrainer(vocab_size=1000,
                                                  special_tokens=special_tokens)
wp_tokenizer.train_from_iterator(train_reviews, wp_trainer)

In [69]:
wp_encoding = wp_tokenizer.encode(some_review)
wp_tokens = wp_encoding.tokens
wp_tokens

['what', 'an', 'aw', '##es', '##ome', 'movie', '!', '<unk>']

In [70]:
wp_token_ids = wp_encoding.ids
wp_token_ids

[443, 312, 635, 257, 354, 331, 4, 1]

In [71]:
wp_decoded = wp_tokenizer.decode(wp_token_ids)
wp_decoded

'what an aw ##es ##ome movie !'

In [72]:
wp_decoded.replace(" ##", "").replace(" !", "!")

'what an awesome movie!'

### Unigram LM

In [73]:
unigram_model = tokenizers.models.Unigram()
unigram_tokenizer = tokenizers.Tokenizer(unigram_model)
unigram_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Whitespace()
unigram_trainer = tokenizers.trainers.UnigramTrainer(
    vocab_size=1000, special_tokens=special_tokens, unk_token="<unk>")
unigram_tokenizer.train_from_iterator(train_reviews, unigram_trainer)

In [74]:
unigram_encoding = unigram_tokenizer.encode(some_review)
unigram_tokens = unigram_encoding.tokens
unigram_tokens

['what', 'an', 'a', 'w', 'e', 'some', 'movie', '!', '😊']

In [75]:
unigram_token_ids = unigram_encoding.ids[:10]
unigram_token_ids

[79, 37, 4, 40, 6, 70, 46, 74, 1]

In [76]:
unigram_tokenizer.decode(unigram_token_ids)

'what an a w e some movie !'

### Pretrained Tokenizers

BBPE is used by models like GPT-2 and RoBERTa.

In [77]:
import transformers

gpt2_tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2")
gpt2_encoding = gpt2_tokenizer(train_reviews[:3], truncation=True,
                               max_length=500)

In [78]:
gpt2_encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [79]:
gpt2_token_ids = gpt2_encoding["input_ids"][0][:10]
gpt2_token_ids

[14247, 35030, 1690, 423, 257, 1688, 8046, 13, 484, 1690]

In [80]:
gpt2_tokenizer.decode(gpt2_token_ids)

'stage adaptations often have a major fault. they often'

WordPiece is used by models like BERT.

In [81]:
bert_tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
bert_encoding = bert_tokenizer(train_reviews[:3], padding=True,
                               truncation=True, max_length=500,
                               return_tensors="pt")

In [82]:
bert_encoding["input_ids"]

tensor([[  101,  2754, 17241,  2411,  2031,  1037,  2350,  6346,  1012,  2027,
          2411,  2272,  2041,  2559,  2066,  1037,  2143,  4950,  2001,  3432,
          2872,  2006,  1996,  2754,  1006,  2107,  2004,  1000,  2305,  2388,
          1000,  1007,  1012, 11430, 11320, 11368,  1005,  1055,  3257,  7906,
          1996,  2143,  4142,  1010,  2029,  2003,  2926,  3697,  2144,  1996,
          3861,  3253,  2032,  2053,  2613,  4119,  1012,  2145,  1010,  2009,
          1005,  1055,  3835,  2000,  2298,  2012,  2005,  2054,  2009,  2003,
          1012,  1996,  6370,  2090,  2745, 19881,  1998,  5696, 20726,  2003,
          3243,  8235,  1012,  1996, 10949,  1997,  2037,  3276,  2024, 11341,
          1012, 19881,  2003, 10392,  2004,  2467,  1010,  1998, 20726,  4152,
          2028,  1997,  2010,  2261,  9592,  2000,  2428,  2552,  1012,  1026,
          7987,  1013,  1028,  1026,  7987,  1013,  1028,  1045, 18766,  2008,
          1045,  1005,  2310,  2196,  2464, 11209, 2

In [83]:
bert_encoding["attention_mask"]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [84]:
# extra code – shows how to drop the special tokens
bert_encoding = bert_tokenizer(train_reviews[:3], padding=True,
                               truncation=True, max_length=500,
                               add_special_tokens=False, return_tensors="pt")
bert_encoding["input_ids"]

tensor([[ 2754, 17241,  2411,  2031,  1037,  2350,  6346,  1012,  2027,  2411,
          2272,  2041,  2559,  2066,  1037,  2143,  4950,  2001,  3432,  2872,
          2006,  1996,  2754,  1006,  2107,  2004,  1000,  2305,  2388,  1000,
          1007,  1012, 11430, 11320, 11368,  1005,  1055,  3257,  7906,  1996,
          2143,  4142,  1010,  2029,  2003,  2926,  3697,  2144,  1996,  3861,
          3253,  2032,  2053,  2613,  4119,  1012,  2145,  1010,  2009,  1005,
          1055,  3835,  2000,  2298,  2012,  2005,  2054,  2009,  2003,  1012,
          1996,  6370,  2090,  2745, 19881,  1998,  5696, 20726,  2003,  3243,
          8235,  1012,  1996, 10949,  1997,  2037,  3276,  2024, 11341,  1012,
         19881,  2003, 10392,  2004,  2467,  1010,  1998, 20726,  4152,  2028,
          1997,  2010,  2261,  9592,  2000,  2428,  2552,  1012,  1026,  7987,
          1013,  1028,  1026,  7987,  1013,  1028,  1045, 18766,  2008,  1045,
          1005,  2310,  2196,  2464, 11209, 20206,  

Unigram LM is used by models like ALBERT, T5, and XLM-R.

In [85]:
albert_tokenizer = transformers.AutoTokenizer.from_pretrained("albert-base-v2")
albert_encoding = albert_tokenizer(
    train_reviews[:3], padding=True, truncation=True, max_length=500,
    add_special_tokens=False, return_tensors="pt")
albert_token_ids = albert_encoding["input_ids"]
albert_token_ids

tensor([[  876,  5004,    18,   478,    57,    21,   394,  4173,     9,    59,
           478,   340,    70,   699,   101,    21,   171,  3336,    23,  1659,
          1037,    27,    14,   876,    13,     5,  4289,    28,    13,     7,
          4893,   449,     7,     6,     9, 12508,  1612,  5909,    22,    18,
          1400,  8968,    14,   171,  2481,    15,    56,    25,  1118,  1956,
           179,    14,  2151,  1434,    61,    90,   683,  2404,     9,   174,
            15,    32,    22,    18,  2210,    20,   361,    35,    26,    98,
            32,    25,     9,    14,  5427,   128,   832, 22427,    17,  4479,
         24604,    25,  1450,  7472,     9,    14, 12289,    16,    66,  1429,
            50, 12891,     9, 22427,    25, 10356,    28,   550,    15,    17,
         24604,  3049,    53,    16,    33,   310, 11285,    20,   510,   601,
             9,     1,  5145,    13,   118,     1,  5145,    13,   118,     1,
            49, 14586,    30,    31,    22,   195,  

In [86]:
hf_tokenizer = transformers.PreTrainedTokenizerFast(
    tokenizer_object=bpe_tokenizer)
hf_encodings = hf_tokenizer(train_reviews[:3], padding=True, truncation=True,
                            max_length=500, return_tensors="pt")
hf_encodings["input_ids"]

tensor([[159, 402, 176, 246,  61, 782, 156, 737, 252,  42, 239,  51, 154, 460,
         917,  17, 272, 156, 737, 576, 215, 976, 275,  42, 199,  44, 554,  42,
         192, 585,  57, 160, 259, 170, 157, 143, 138, 159, 402,  11, 589, 152,
           5, 819, 168, 230,   5, 521, 924, 981, 962, 250,  61,  10,  60, 426,
         526, 959,  60, 138, 199, 150, 319,  15, 363, 141, 957, 694,  47, 696,
          61, 875, 138, 960, 337, 414, 140, 157, 385, 174, 433, 161, 221, 145,
         213,  17, 549,  15, 151,  10,  60,  55, 416, 146, 407, 144, 182, 303,
         151, 141,  17, 138, 547, 538, 528, 768,  54, 335,  42, 203,  44, 270,
          46, 153, 876, 141, 919, 233, 522, 172, 141, 719, 162, 807, 279,  17,
         138,  45,  66,  55, 188, 989, 156, 378, 698, 301, 296, 689, 212, 558,
         926, 148,  17,  44, 270,  46, 141,  47, 279, 302, 171, 152, 787,  15,
         153, 522, 172, 766, 205, 156, 234, 677, 161, 139, 513, 146, 370, 251,
         219, 162, 197, 162, 166,  50, 265,  47, 266

## Building and Training a Sentiment Analysis Model

In [87]:
def collate_fn(batch, tokenizer=bert_tokenizer):
    reviews = [review["text"] for review in batch]
    labels = [[review["label"]] for review in batch]
    encodings = tokenizer(reviews, padding=True, truncation=True,
                          max_length=200, return_tensors="pt")
    labels = torch.tensor(labels, dtype=torch.float32)
    return encodings, labels

batch_size = 256
imdb_train_loader = DataLoader(imdb_train_set, batch_size=batch_size,
                               collate_fn=collate_fn, shuffle=True)
imdb_valid_loader = DataLoader(imdb_valid_set, batch_size=batch_size,
                               collate_fn=collate_fn)
imdb_test_loader = DataLoader(imdb_test_set, batch_size=batch_size,
                              collate_fn=collate_fn)

In [88]:
class SentimentAnalysisModel(nn.Module):
    def __init__(self, vocab_size, n_layers=2, embedding_size=128,
                 hidden_size=64, pad_id=0, dropout=0.2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size,
                                  padding_idx=pad_id)
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers=n_layers,
                          batch_first=True, dropout=dropout)
        self.output = nn.Linear(hidden_size, 1)

    def forward(self, encodings):
        embeddings = self.embed(encodings["input_ids"])
        _outputs, hidden_states = self.gru(embeddings)
        return self.output(hidden_states[-1])

In [89]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

sequences = torch.tensor([[1, 2, 0, 0], [5, 6, 7, 8]])
packed = pack_padded_sequence(sequences, lengths=(2, 4),
                              enforce_sorted=False, batch_first=True)
packed

PackedSequence(data=tensor([5, 1, 6, 2, 7, 8]), batch_sizes=tensor([2, 2, 1, 1]), sorted_indices=tensor([1, 0]), unsorted_indices=tensor([1, 0]))

In [90]:
padded, lengths = pad_packed_sequence(packed, batch_first=True)
padded, lengths

(tensor([[1, 2, 0, 0],
         [5, 6, 7, 8]]),
 tensor([2, 4]))

In [91]:
class SentimentAnalysisModelPackedSeq(nn.Module):
    def __init__(self, vocab_size, n_layers=2, embedding_size=128,
                 hidden_size=64, pad_id=0, dropout=0.2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size,
                                  padding_idx=pad_id)
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers=n_layers,
                          batch_first=True, dropout=dropout)
        self.output = nn.Linear(hidden_size, 1)

    def forward(self, encodings):
        embeddings = self.embed(encodings["input_ids"])
        lengths = encodings["attention_mask"].sum(dim=1)                      # <= line added
        packed = pack_padded_sequence(embeddings, lengths=lengths.cpu(),      # <= line added
                                      batch_first=True, enforce_sorted=False) # <= line added
        _outputs, hidden_states = self.gru(packed)                            # <= line changed
        return self.output(hidden_states[-1])

In [92]:
torch.manual_seed(42)

vocab_size = bert_tokenizer.vocab_size
imdb_model_ps = SentimentAnalysisModelPackedSeq(vocab_size).to(device)

n_epochs = 10
xentropy = nn.BCEWithLogitsLoss()
optimizer = torch.optim.NAdam(imdb_model_ps.parameters())
accuracy = torchmetrics.Accuracy(task="binary").to(device)

history = train(imdb_model_ps, optimizer, xentropy, accuracy,
                imdb_train_loader, imdb_valid_loader, n_epochs)

Epoch 1/10,                      train loss: 0.6640, train metric: 59.20%, valid metric: 60.30%
Epoch 2/10,                      train loss: 0.5749, train metric: 70.63%, valid metric: 65.98%
Epoch 3/10,                      train loss: 0.4963, train metric: 76.87%, valid metric: 65.38%
Epoch 4/10,                      train loss: 0.3539, train metric: 84.68%, valid metric: 82.78%
Epoch 5/10,                      train loss: 0.2662, train metric: 89.26%, valid metric: 83.76%
Epoch 6/10,                      train loss: 0.1985, train metric: 92.58%, valid metric: 85.14%
Epoch 7/10,                      train loss: 0.1528, train metric: 94.55%, valid metric: 85.54%
Epoch 8/10,                      train loss: 0.1168, train metric: 96.21%, valid metric: 85.20%
Epoch 9/10,                      train loss: 0.0637, train metric: 98.22%, valid metric: 83.12%
Epoch 10/10,                      train loss: 0.0672, train metric: 98.37%, valid metric: 81.68%


In [93]:
class SentimentAnalysisModelBidi(nn.Module):
    def __init__(self, vocab_size, n_layers=2, embedding_size=128,
                 hidden_size=64, pad_id=0, dropout=0.2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size,
                                  padding_idx=pad_id)
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers=n_layers,
                          batch_first=True, dropout=dropout, bidirectional=True)  # <= line changed
        self.output = nn.Linear(2 * hidden_size, 1)                               # <= line changed

    def forward(self, encodings):
        embeddings = self.embed(encodings["input_ids"])
        lengths = encodings["attention_mask"].sum(dim=1)
        packed = pack_padded_sequence(embeddings, lengths=lengths.cpu(),
                                      batch_first=True, enforce_sorted=False)
        _outputs, hidden_states = self.gru(packed)
        n_dims = self.output.in_features                                          # <= line added
        top_states = hidden_states[-2:].permute(1, 0, 2).reshape(-1, n_dims)      # <= line added
        return self.output(top_states)                                            # <= line changed

In [94]:
torch.manual_seed(42)

vocab_size = bert_tokenizer.vocab_size
imdb_model_bidi = SentimentAnalysisModelBidi(vocab_size).to(device)

n_epochs = 10
xentropy = nn.BCEWithLogitsLoss()
optimizer = torch.optim.NAdam(imdb_model_bidi.parameters())
accuracy = torchmetrics.Accuracy(task="binary").to(device)

history = train(imdb_model_bidi, optimizer, xentropy, accuracy, imdb_train_loader,
                imdb_valid_loader, n_epochs)

Epoch 1/10,                      train loss: 0.6528, train metric: 60.66%, valid metric: 72.78%
Epoch 2/10,                      train loss: 0.5046, train metric: 75.52%, valid metric: 77.32%
Epoch 3/10,                      train loss: 0.3865, train metric: 82.78%, valid metric: 80.82%
Epoch 4/10,                      train loss: 0.2942, train metric: 87.85%, valid metric: 81.86%
Epoch 5/10,                      train loss: 0.2445, train metric: 90.20%, valid metric: 73.08%
Epoch 6/10,                      train loss: 0.1561, train metric: 94.13%, valid metric: 83.76%
Epoch 7/10,                      train loss: 0.1020, train metric: 96.36%, valid metric: 83.76%
Epoch 8/10,                      train loss: 0.0496, train metric: 98.42%, valid metric: 79.04%
Epoch 9/10,                      train loss: 0.0714, train metric: 97.77%, valid metric: 82.76%
Epoch 10/10,                      train loss: 0.0180, train metric: 99.66%, valid metric: 82.54%


## Reusing Pretrained Embeddings and Language Models

In [95]:
bert_model = transformers.AutoModel.from_pretrained("bert-base-uncased")
bert_model.embeddings.word_embeddings

Embedding(30522, 768, padding_idx=0)

In [96]:
class SentimentAnalysisModelPreEmbeds(nn.Module):
    def __init__(self, pretrained_embeddings, n_layers=2, hidden_size=64,
                 dropout=0.2):
        super().__init__()
        weights = pretrained_embeddings.weight.data
        self.embed = nn.Embedding.from_pretrained(weights, freeze=True)
        embedding_size = weights.shape[-1]
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers=n_layers,
                          batch_first=True, dropout=dropout, bidirectional=True)
        self.output = nn.Linear(2 * hidden_size, 1)

    def forward(self, encodings):
        embeddings = self.embed(encodings["input_ids"])
        lengths = encodings["attention_mask"].sum(dim=1)
        packed = pack_padded_sequence(embeddings, lengths=lengths.cpu(),
                                      batch_first=True, enforce_sorted=False)
        _outputs, hidden_states = self.gru(packed)
        n_dims = self.output.in_features
        top_states = hidden_states[-2:].permute(1, 0, 2).reshape(-1, n_dims)
        return self.output(top_states)

In [97]:
torch.manual_seed(42)

imdb_model_bert_embeds = SentimentAnalysisModelPreEmbeds(
    bert_model.embeddings.word_embeddings).to(device)

n_epochs = 10
xentropy = nn.BCEWithLogitsLoss()
optimizer = torch.optim.NAdam(imdb_model_bert_embeds.parameters())
accuracy = torchmetrics.Accuracy(task="binary").to(device)

history = train(imdb_model_bert_embeds, optimizer, xentropy, accuracy,
                imdb_train_loader, imdb_valid_loader, n_epochs)

Epoch 1/10,                      train loss: 0.6918, train metric: 55.10%, valid metric: 52.20%
Epoch 2/10,                      train loss: 0.6499, train metric: 63.81%, valid metric: 71.42%
Epoch 3/10,                      train loss: 0.5545, train metric: 72.01%, valid metric: 72.48%
Epoch 4/10,                      train loss: 0.4817, train metric: 77.03%, valid metric: 78.92%
Epoch 5/10,                      train loss: 0.4143, train metric: 80.99%, valid metric: 78.60%
Epoch 6/10,                      train loss: 0.3678, train metric: 83.38%, valid metric: 83.78%
Epoch 7/10,                      train loss: 0.3337, train metric: 85.26%, valid metric: 83.62%
Epoch 8/10,                      train loss: 0.3050, train metric: 86.92%, valid metric: 82.52%
Epoch 9/10,                      train loss: 0.2856, train metric: 87.84%, valid metric: 68.18%
Epoch 10/10,                      train loss: 0.2435, train metric: 90.06%, valid metric: 82.28%


In [98]:
bert_encoding = bert_tokenizer(train_reviews[:3], padding=True,
                               max_length=200, truncation=True,
                               return_tensors="pt")
bert_output = bert_model(**bert_encoding)
bert_output.last_hidden_state.shape

torch.Size([3, 200, 768])

In [99]:
bert_output.pooler_output.shape

torch.Size([3, 768])

In [100]:
class SentimentAnalysisModelBert(nn.Module):
    def __init__(self, n_layers=2, hidden_size=64, dropout=0.2):
        super().__init__()
        self.bert = transformers.AutoModel.from_pretrained("bert-base-uncased")
        embedding_size = self.bert.config.hidden_size
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers=n_layers,
                          batch_first=True, dropout=dropout)
        self.output = nn.Linear(hidden_size, 1)

    def forward(self, encodings):
        contextualized_embeddings = self.bert(**encodings).last_hidden_state
        lengths = encodings["attention_mask"].sum(dim=1)
        packed = pack_padded_sequence(contextualized_embeddings, lengths=lengths.cpu(),
                                      batch_first=True, enforce_sorted=False)
        _outputs, hidden_states = self.gru(packed)
        return self.output(hidden_states[-1])

In [101]:
torch.manual_seed(42)

imdb_model_bert = SentimentAnalysisModelBert().to(device)
imdb_model_bert.bert.requires_grad_(False)

n_epochs = 4
xentropy = nn.BCEWithLogitsLoss()
optimizer = torch.optim.NAdam(imdb_model_bert.parameters())
accuracy = torchmetrics.Accuracy(task="binary").to(device)

history = train(imdb_model_bert, optimizer, xentropy, accuracy,
                imdb_train_loader, imdb_valid_loader, n_epochs)

Epoch 1/4,                      train loss: 0.4860, train metric: 75.67%, valid metric: 87.02%
Epoch 2/4,                      train loss: 0.3024, train metric: 87.01%, valid metric: 88.80%
Epoch 3/4,                      train loss: 0.2728, train metric: 88.68%, valid metric: 87.60%
Epoch 4/4,                      train loss: 0.2531, train metric: 89.46%, valid metric: 86.68%


In [102]:
class SentimentAnalysisModelBert2(nn.Module):
    def __init__(self, hidden_size=64):
        super().__init__()
        self.bert = transformers.AutoModel.from_pretrained("bert-base-uncased")
        self.output = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, encodings):
        bert_output = self.bert(**encodings)
        return self.output(bert_output.last_hidden_state[:, 0])

In [103]:
class SentimentAnalysisModelBert3(nn.Module):
    def __init__(self, hidden_size=64):
        super().__init__()
        self.bert = transformers.AutoModel.from_pretrained("bert-base-uncased")
        self.output = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, encodings):
        bert_output = self.bert(**encodings)
        return self.output(bert_output.pooler_output)

In [104]:
torch.manual_seed(42)

imdb_model_bert3 = SentimentAnalysisModelBert3().to(device)
imdb_model_bert3.bert.requires_grad_(False)

n_epochs = 5
xentropy = nn.BCEWithLogitsLoss()
optimizer = torch.optim.NAdam(imdb_model_bert3.parameters())
accuracy = torchmetrics.Accuracy(task="binary").to(device)

history = train(imdb_model_bert3, optimizer, xentropy, accuracy,
                imdb_train_loader, imdb_valid_loader, n_epochs)

Epoch 1/5,                      train loss: 0.6593, train metric: 61.35%, valid metric: 67.72%
Epoch 2/5,                      train loss: 0.6035, train metric: 69.63%, valid metric: 68.34%
Epoch 3/5,                      train loss: 0.5710, train metric: 72.01%, valid metric: 74.64%
Epoch 4/5,                      train loss: 0.5540, train metric: 73.47%, valid metric: 71.56%
Epoch 5/5,                      train loss: 0.5365, train metric: 74.22%, valid metric: 59.56%


In [105]:
imdb_model_bert3.bert.pooler.requires_grad_(True)

history = train(imdb_model_bert3, optimizer, xentropy, accuracy,
                imdb_train_loader, imdb_valid_loader, n_epochs)

Epoch 1/5,                      train loss: 0.8118, train metric: 71.17%, valid metric: 81.50%
Epoch 2/5,                      train loss: 0.4543, train metric: 79.32%, valid metric: 81.00%
Epoch 3/5,                      train loss: 0.4418, train metric: 79.50%, valid metric: 80.36%
Epoch 4/5,                      train loss: 0.4192, train metric: 80.70%, valid metric: 77.80%
Epoch 5/5,                      train loss: 0.4038, train metric: 81.81%, valid metric: 82.68%


# Task-Specific Classes

In [106]:
from transformers import AutoModelForSequenceClassification

torch.manual_seed(42)
bert_for_binary_clf = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [107]:
encoding = bert_tokenizer(["This was a great movie!"])
output = bert_for_binary_clf(
    input_ids=torch.tensor(encoding["input_ids"]),
    attention_mask=torch.tensor(encoding["attention_mask"]))

output.logits

tensor([[0.2468, 0.3499]], grad_fn=<AddmmBackward0>)

In [108]:
torch.softmax(output.logits, dim=-1)

tensor([[0.4743, 0.5257]], grad_fn=<SoftmaxBackward0>)

In [109]:
output = bert_for_binary_clf(
    input_ids=torch.tensor(encoding["input_ids"]),
    attention_mask=torch.tensor(encoding["attention_mask"]),
    labels=torch.tensor([1]))

output.loss

tensor(0.6429, grad_fn=<NllLossBackward0>)

Every model from the Transformers library has a `config` attribute that contains the model's configuration:

In [110]:
bert_for_binary_clf.config

BertConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

# The Trainer API

We could use our own training function to train this model, or we can use the Trainer API instead.

The Trainer API needs datasets that are already tokenized, so let's prepare them:

In [111]:
def tokenize_batch(batch):
    return bert_tokenizer(batch["text"], truncation=True, max_length=200)

tok_imdb_train_set = imdb_train_set.map(tokenize_batch, batched=True)
tok_imdb_valid_set = imdb_valid_set.map(tokenize_batch, batched=True)
tok_imdb_test_set = imdb_test_set.map(tokenize_batch, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

The Trainer API doesn't support streaming metrics so we need to write our own evaluation function:

In [112]:
def compute_accuracy(pred):
    return {"accuracy": (pred.label_ids == pred.predictions.argmax(-1)).mean()}

In [113]:
from transformers import TrainingArguments

train_args = TrainingArguments(
    output_dir="my_imdb_model", num_train_epochs=2,
    per_device_train_batch_size=128, per_device_eval_batch_size=128,
    eval_strategy="epoch", logging_strategy="epoch", save_strategy="epoch",
    load_best_model_at_end=True, metric_for_best_model="accuracy")

In [114]:
from transformers import DataCollatorWithPadding, Trainer

trainer = Trainer(
    bert_for_binary_clf, train_args, train_dataset=tok_imdb_train_set,
    eval_dataset=tok_imdb_valid_set, compute_metrics=compute_accuracy,
    data_collator=DataCollatorWithPadding(bert_tokenizer))
train_output = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.305000,0.241851,0.902000
2,0.161500,0.242788,0.907000


# Pipelines

In [115]:
from transformers import pipeline

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
classifier_imdb = pipeline("sentiment-analysis", model=model_name,
                           truncation=True, max_length=512)
classifier_imdb(train_reviews[:10])

Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.9996108412742615},
 {'label': 'POSITIVE', 'score': 0.9998623132705688},
 {'label': 'NEGATIVE', 'score': 0.9943684935569763},
 {'label': 'POSITIVE', 'score': 0.997913658618927},
 {'label': 'POSITIVE', 'score': 0.999544084072113},
 {'label': 'NEGATIVE', 'score': 0.9845332503318787},
 {'label': 'POSITIVE', 'score': 0.9859278202056885},
 {'label': 'POSITIVE', 'score': 0.9993758797645569},
 {'label': 'POSITIVE', 'score': 0.9978922009468079},
 {'label': 'NEGATIVE', 'score': 0.9997020363807678}]

In [116]:
accuracy = torchmetrics.Accuracy(task="binary").to(device)
with torch.no_grad():
    text_imdb_valid_loader = DataLoader(imdb_valid_set, batch_size=256)
    for index, batch in enumerate(text_imdb_valid_loader):
        y_pred = classifier_imdb(batch["text"], truncation=True)
        y_pred = torch.tensor([pred["label"] == "POSITIVE" for pred in y_pred], dtype=int)
        accuracy.update(y_pred, batch["label"])
        print(f"\r{index + 1}/{len(text_imdb_valid_loader)}", end="")

accuracy.compute()

20/20

tensor(0.8820, device='mps:0')

Models can be very biased. For example, it may like or dislike some countries depending on the data it was trained on, and how it is used, so use it with care:

In [117]:
# extra code – shows that binary classification can amplify model bias
countries = ["Iraq", "Thailand", "the USA", "Vietnam"]
texts = [f"I am from {country}" for country in countries]
list(zip(countries, classifier_imdb(texts)))

[('Iraq', {'label': 'NEGATIVE', 'score': 0.9706069231033325}),
 ('Thailand', {'label': 'POSITIVE', 'score': 0.9903932213783264}),
 ('the USA', {'label': 'POSITIVE', 'score': 0.9642282128334045}),
 ('Vietnam', {'label': 'NEGATIVE', 'score': 0.9747399091720581})]

In [118]:
# extra code – using a model with a neutral class solves this bias issue
# note: the warning is normal: this model's pooler will not be used for
# classification, so its weights are downloaded but not used.
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
classifier_imdb_with_neutral = pipeline("sentiment-analysis", model=model_name)
list(zip(countries, classifier_imdb_with_neutral(texts)))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


[('Iraq', {'label': 'neutral', 'score': 0.8353288769721985}),
 ('Thailand', {'label': 'neutral', 'score': 0.8824350237846375}),
 ('the USA', {'label': 'neutral', 'score': 0.8349123597145081}),
 ('Vietnam', {'label': 'neutral', 'score': 0.8436853885650635})]

In [119]:
model_name = "huggingface/distilbert-base-uncased-finetuned-mnli"
classifier_mnli = pipeline("text-classification", model=model_name)
classifier_mnli([
    "She loves me. [SEP] She loves me not. [SEP]",
    "Alice just woke up. [SEP] Alice is awake. [SEP]",
    "I like dogs. [SEP] Everyone likes dogs. [SEP]"])

Device set to use mps:0


[{'label': 'contradiction', 'score': 0.9717152714729309},
 {'label': 'entailment', 'score': 0.9119168519973755},
 {'label': 'neutral', 'score': 0.9509280323982239}]

# An Encoder–Decoder Network for Neural Machine Translation

In [120]:
nmt_dataset = load_dataset("Helsinki-NLP/tatoeba_mt", language_pair="eng-spa",
                           trust_remote_code=True)  # only if you trust the user
split = nmt_dataset["validation"].train_test_split(train_size=0.8, seed=42)
nmt_train_set, nmt_valid_set = split["train"], split["test"]
nmt_test_set = nmt_dataset["test"]

In [121]:
nmt_train_set[0]

{'sourceLang': 'eng',
 'targetlang': 'spa',
 'sourceString': "Turn the light off. I can't fall asleep.",
 'targetString': 'Apaga la luz. No me puedo dormir.'}

In [122]:
def train_eng_spa():  # a generator function to iterate over all training text
    for pair in nmt_train_set:
        yield pair["sourceString"]
        yield pair["targetString"]

max_length = 500
vocab_size = 10_000

nmt_tokenizer_model = tokenizers.models.BPE(unk_token="<unk>")
nmt_tokenizer = tokenizers.Tokenizer(nmt_tokenizer_model)
nmt_tokenizer.enable_padding(pad_id=0, pad_token="<pad>")
nmt_tokenizer.enable_truncation(max_length=max_length)
nmt_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Whitespace()
nmt_tokenizer_trainer = tokenizers.trainers.BpeTrainer(
    vocab_size=vocab_size, special_tokens=["<pad>", "<unk>", "<s>", "</s>"])
nmt_tokenizer.train_from_iterator(train_eng_spa(), nmt_tokenizer_trainer)

In [123]:
nmt_tokenizer.encode("I like soccer").ids

[43, 403, 4300]

In [124]:
nmt_tokenizer.encode("<s> Me gusta el fútbol").ids

[2, 397, 583, 221, 3325]

In [125]:
from collections import namedtuple

fields = ["src_token_ids", "src_mask", "tgt_token_ids", "tgt_mask"]
class NmtPair(namedtuple("NmtPairBase", fields)):
    def to(self, device):
        return NmtPair(self.src_token_ids.to(device), self.src_mask.to(device), 
                       self.tgt_token_ids.to(device), self.tgt_mask.to(device))

In [126]:
def nmt_collate_fn(batch):
    src_texts = [pair['sourceString'] for pair in batch]
    tgt_texts = [f"<s> {pair['targetString']} </s>" for pair in batch]
    src_encodings = nmt_tokenizer.encode_batch(src_texts)
    tgt_encodings = nmt_tokenizer.encode_batch(tgt_texts)
    src_token_ids = torch.tensor([enc.ids for enc in src_encodings])
    tgt_token_ids = torch.tensor([enc.ids for enc in tgt_encodings])
    src_mask = torch.tensor([enc.attention_mask for enc in src_encodings])
    tgt_mask = torch.tensor([enc.attention_mask for enc in tgt_encodings])
    inputs = NmtPair(src_token_ids, src_mask,
                     tgt_token_ids[:, :-1], tgt_mask[:, :-1])
    labels = tgt_token_ids[:, 1:]
    return inputs, labels

batch_size = 256
nmt_train_loader = DataLoader(nmt_train_set, batch_size=batch_size,
                              collate_fn=nmt_collate_fn, shuffle=True)
nmt_valid_loader = DataLoader(nmt_valid_set, batch_size=batch_size,
                              collate_fn=nmt_collate_fn)
nmt_test_loader = DataLoader(nmt_test_set, batch_size=batch_size,
                             collate_fn=nmt_collate_fn)

In [127]:
class NmtModel(nn.Module):
    def __init__(self, vocab_size, embed_size=512, pad_id=0, hidden_size=512,
                 n_layers=2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=pad_id)
        self.encoder = nn.GRU(embed_size, hidden_size, num_layers=n_layers,
                              batch_first=True)
        self.decoder = nn.GRU(embed_size, hidden_size, num_layers=n_layers,
                              batch_first=True)
        self.output = nn.Linear(hidden_size, vocab_size)

    def forward(self, pair):
        src_embeddings = self.embed(pair.src_token_ids)
        tgt_embeddings = self.embed(pair.tgt_token_ids)
        src_lengths = pair.src_mask.sum(dim=1)
        src_packed = pack_padded_sequence(
            src_embeddings, lengths=src_lengths.cpu(),
            batch_first=True, enforce_sorted=False)
        _, hidden_states = self.encoder(src_packed)
        outputs, _ = self.decoder(tgt_embeddings, hidden_states)
        return self.output(outputs).permute(0, 2, 1)

torch.manual_seed(42)
vocab_size = nmt_tokenizer.get_vocab_size()
nmt_model = NmtModel(vocab_size).to(device)

In [128]:
n_epochs = 20
xentropy = nn.CrossEntropyLoss(ignore_index=0)  # ignore <pad> tokens
optimizer = torch.optim.NAdam(nmt_model.parameters())
accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=vocab_size)
accuracy = accuracy.to(device)

history = train(nmt_model, optimizer, xentropy, accuracy,
                nmt_train_loader, nmt_valid_loader, n_epochs)

Epoch 1/20,                      train loss: 3.4533, train metric: 8.26%, valid metric: 10.13%
Epoch 2/20,                      train loss: 2.0985, train metric: 11.29%, valid metric: 10.98%
Epoch 3/20,                      train loss: 1.6588, train metric: 12.25%, valid metric: 11.19%
Epoch 4/20,                      train loss: 1.4099, train metric: 13.05%, valid metric: 11.24%
Epoch 5/20,                      train loss: 1.2552, train metric: 13.34%, valid metric: 11.21%
Epoch 6/20,                      train loss: 1.1415, train metric: 13.78%, valid metric: 11.20%
Epoch 7/20,                      train loss: 1.0622, train metric: 13.99%, valid metric: 11.14%
Epoch 8/20,                      train loss: 0.7658, train metric: 15.24%, valid metric: 11.47%
Epoch 9/20,                      train loss: 0.5246, train metric: 16.29%, valid metric: 11.46%
Epoch 10/20,                      train loss: 0.4120, train metric: 16.76%, valid metric: 11.43%
Epoch 11/20,                      train 

In [129]:
torch.save(nmt_model.state_dict(), "my_nmt_model.pt")

In [130]:
def translate(model, src_text, max_length=20, pad_id=0, eos_id=3):
    tgt_text = ""
    token_ids = []
    for index in range(max_length):
        batch, _ = nmt_collate_fn([{"sourceString": src_text,
                                    "targetString": tgt_text}])
        with torch.no_grad():
            Y_logits = model(batch.to(device))
            Y_token_ids = Y_logits.argmax(dim=1)  # find the best token IDs
            next_token_id = Y_token_ids[0, index]  # take the last token ID

        next_token = nmt_tokenizer.id_to_token(next_token_id)
        tgt_text += " " + next_token
        if next_token_id == eos_id:
            break
    return tgt_text

In [131]:
nmt_model.eval()
translate(nmt_model, "I like soccer.")

' Me gusta el fútbol . </s>'

In [132]:
longer_text = "I like to play soccer with my friends."
translate(nmt_model, longer_text)

' Me gusta jugar con mis amigos . </s>'

## Beam Search

This is a very basic implementation of beam search. I tried to make it readable and understandable, but it's definitely not optimized for speed! The function first uses the model to find the top _k_ words to start the translations (where _k_ is the beam width). For each of the top _k_ translations, it evaluates the conditional probabilities of all possible words it could add to that translation. These extended translations and their probabilities are added to the list of candidates. Once we've gone through all top _k_ translations and all words that could complete them, we keep only the top _k_ candidates with the highest probability, and we iterate over and over until they all finish with an EOS token. The top translation is then returned (after removing its EOS token).

* Note: If p(S) is the probability of sentence S, and p(W|S) is the conditional probability of the word W given that the translation starts with S, then the probability of the sentence S' = concat(S, W) is p(S') = p(S) * p(W|S). As we add more words, the probability gets smaller and smaller. To avoid the risk of it getting too small, which could cause floating point precision errors, the function keeps track of log probabilities instead of probabilities: recall that log(a\*b) = log(a) + log(b), therefore log(p(S')) = log(p(S)) + log(p(W|S)).

In [133]:
def beam_search(model, src_text, beam_width=3, max_length=20,
                verbose=False, length_penalty=0.6):
    top_translations = [(torch.tensor(0.), "")]
    for index in range(max_length):
        if verbose:
            print(f"Top {beam_width} translations so far:")
            for log_proba, tgt_text in top_translations:
                print(f"    {log_proba.item():.3f} – {tgt_text}")

        candidates = []
        for log_proba, tgt_text in top_translations:
            if tgt_text.endswith(" </s>"):
                candidates.append((log_proba, tgt_text))
                continue  # don't add tokens after EOS token
            batch, _ = nmt_collate_fn([{"sourceString": src_text,
                                        "targetString": tgt_text}])
            with torch.no_grad():
                Y_logits = model(batch.to(device))
                Y_log_proba = F.log_softmax(Y_logits, dim=1)
                Y_top_log_probas = torch.topk(Y_log_proba, k=beam_width, dim=1)

            for beam_index in range(beam_width):
                next_token_log_proba = Y_top_log_probas.values[0, beam_index, index]
                next_token_id = Y_top_log_probas.indices[0, beam_index, index]
                next_token = nmt_tokenizer.id_to_token(next_token_id)
                next_tgt_text = tgt_text + " " + next_token
                candidates.append((log_proba + next_token_log_proba, next_tgt_text))

        def length_penalized_score(candidate, alpha=length_penalty):
            log_proba, text = candidate
            length = len(text.split())
            penalty = ((5 + length) ** alpha) / (6 ** alpha)
            return log_proba / penalty

        top_translations = sorted(candidates,
                                  key=length_penalized_score,
                                  reverse=True)[:beam_width]

    return top_translations[-1][1]

In [134]:
beam_search(nmt_model, longer_text, beam_width=3)

' Me gusta jugar al fútbol con mis amigos . </s>'

In [135]:
longest_text = "I like to play soccer with my friends at the beach."
beam_search(nmt_model, longest_text, beam_width=3)

' Me gusta jugar con jugar con los jug adores de la playa . </s>'

# Attention Mechanisms

Let's implement Luong attention (a.k.a. dot-product attention):

In [136]:
def attention(query, key, value):
    scores = query @ key.transpose(1, 2)  # [B, Lq, D] @ [B, D, Lk]= [B, Lq, Lk]
    weights = torch.softmax(scores, dim=-1)  # [B, Lq, Lk]
    return weights @ value  # [B, Lq, Lk] @ [B, Lk, Dv] = [B, Lq, Dv]

* `B` = batch size
* `Lq` = max query length in this batch
* `Lk` = max key/value length in this batch (keys and values have the same lengths)
* `D` = query embedding size = key embedding size
* `Dv` = value embedding size

In [137]:
class NmtModelWithAttention(nn.Module):
    def __init__(self, vocab_size, embed_size=512, pad_id=0, hidden_size=512,
                 n_layers=2):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=pad_id)
        self.encoder = nn.GRU(
            embed_size, hidden_size, num_layers=n_layers, batch_first=True)
        self.decoder = nn.GRU(
            embed_size, hidden_size, num_layers=n_layers, batch_first=True)
        self.output = nn.Linear(2 * hidden_size, vocab_size)

    def forward(self, pair):
        src_embeddings = self.embed(pair.src_token_ids)  # same as earlier
        tgt_embeddings = self.embed(pair.tgt_token_ids)  # same
        src_lengths = pair.src_mask.sum(dim=1)  # same
        src_packed = pack_padded_sequence(
            src_embeddings, lengths=src_lengths.cpu(),
            batch_first=True, enforce_sorted=False)  # same
        encoder_outputs_packed, hidden_states = self.encoder(src_packed)
        decoder_outputs, _ = self.decoder(tgt_embeddings, hidden_states)  # same
        encoder_outputs, _ = pad_packed_sequence(encoder_outputs_packed,
                                                 batch_first=True)
        attn_output = attention(query=decoder_outputs,
                                key=encoder_outputs, value=encoder_outputs)
        combined_output = torch.cat((attn_output, decoder_outputs), dim=-1)
        return self.output(combined_output).permute(0, 2, 1)

In [138]:
torch.manual_seed(42)
nmt_attn_model = NmtModelWithAttention(vocab_size).to(device)

n_epochs = 20
xentropy = nn.CrossEntropyLoss(ignore_index=0)  # ignore <pad> tokens
optimizer = torch.optim.NAdam(nmt_attn_model.parameters())
accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=vocab_size)
accuracy = accuracy.to(device)

history = train(nmt_attn_model, optimizer, xentropy, accuracy,
                nmt_train_loader, nmt_valid_loader, n_epochs)

Epoch 1/20,                      train loss: 3.1834, train metric: 8.93%, valid metric: 10.45%
Epoch 2/20,                      train loss: 2.0575, train metric: 11.37%, valid metric: 11.12%
Epoch 3/20,                      train loss: 1.7671, train metric: 12.01%, valid metric: 11.26%
Epoch 4/20,                      train loss: 1.6236, train metric: 12.41%, valid metric: 11.30%
Epoch 5/20,                      train loss: 1.5353, train metric: 12.56%, valid metric: 11.35%
Epoch 6/20,                      train loss: 1.4763, train metric: 12.83%, valid metric: 11.32%
Epoch 7/20,                      train loss: 1.4365, train metric: 12.87%, valid metric: 11.31%
Epoch 8/20,                      train loss: 1.4003, train metric: 13.10%, valid metric: 11.32%
Epoch 9/20,                      train loss: 1.1272, train metric: 13.76%, valid metric: 11.75%
Epoch 10/20,                      train loss: 0.9114, train metric: 14.68%, valid metric: 11.84%
Epoch 11/20,                      train 

In [139]:
torch.save(nmt_attn_model.state_dict(), "my_nmt_attn_model.pt")

In [140]:
translate(nmt_attn_model, longest_text)

' Me gusta jugar fu tbol con mis amigos en la playa . </s>'

# Exercise solutions

## 1. to 6.

1. Stateless RNNs can only capture patterns whose length is less than, or equal to, the size of the windows the RNN is trained on. Conversely, stateful RNNs can capture longer-term patterns. However, implementing a stateful RNN is much harder⁠—especially preparing the dataset properly. Moreover, stateful RNNs do not always work better, in part because consecutive batches are not independent and identically distributed (IID). Gradient Descent is not fond of non-IID datasets.
2. In general, if you translate a sentence one word at a time, the result will be terrible. For example, the French sentence "Je vous en prie" means "You are welcome," but if you translate it one word at a time, you get "I you in pray." Huh? It is much better to read the whole sentence first and then translate it. A plain sequence-to-sequence RNN would start translating a sentence immediately after reading the first word, while an Encoder–Decoder RNN will first read the whole sentence and then translate it. That said, one could imagine a plain sequence-to-sequence RNN that would output silence whenever it is unsure about what to say next (just like human translators do when they must translate a live broadcast).
3. Variable-length input sequences can be handled by padding the shorter sequences so that all sequences in a batch have the same length, and using masking to ensure the RNN ignores the padding token. For better performance, you may also want to create batches containing sequences of similar sizes. Ragged tensors can hold sequences of variable lengths, and Keras now supports them, which simplifies handling variable-length input sequences (at the time of this writing, it still does not handle ragged tensors as targets on the GPU, though). Regarding variable-length output sequences, if the length of the output sequence is known in advance (e.g., if you know that it is the same as the input sequence), then you just need to configure the loss function so that it ignores tokens that come after the end of the sequence. Similarly, the code that will use the model should ignore tokens beyond the end of the sequence. But generally the length of the output sequence is not known ahead of time, so the solution is to train the model so that it outputs an end-of-sequence token at the end of each sequence.
4. Beam search is a technique used to improve the performance of a trained Encoder–Decoder model, for example in a neural machine translation system. The algorithm keeps track of a short list of the _k_ most promising output sentences (say, the top three), and at each decoder step it tries to extend them by one word; then it keeps only the _k_ most likely sentences. The parameter _k_ is called the _beam width_: the larger it is, the more CPU and RAM will be used, but also the more accurate the system will be. Instead of greedily choosing the most likely next word at each step to extend a single sentence, this technique allows the system to explore several promising sentences simultaneously. Moreover, this technique lends itself well to parallelization. You can implement beam search by writing a custom memory cell. Alternatively, TensorFlow Addons's seq2seq API provides an implementation.
5. An attention mechanism is a technique initially used in Encoder–Decoder models to give the decoder more direct access to the input sequence, allowing it to deal with longer input sequences. At each decoder time step, the current decoder's state and the full output of the encoder are processed by an alignment model that outputs an alignment score for each input time step. This score indicates which part of the input is most relevant to the current decoder time step. The weighted sum of the encoder output (weighted by their alignment score) is then fed to the decoder, which produces the next decoder state and the output for this time step. The main benefit of using an attention mechanism is the fact that the Encoder–Decoder model can successfully process longer input sequences. Another benefit is that the alignment scores make the model easier to debug and interpret: for example, if the model makes a mistake, you can look at which part of the input it was paying attention to, and this can help diagnose the issue. An attention mechanism is also at the core of the Transformer architecture, in the Multi-Head Attention layers. See the next answer.
6. Sampled softmax is used when training a classification model when there are many classes (e.g., thousands). It computes an approximation of the cross-entropy loss based on the logit predicted by the model for the correct class, and the predicted logits for a sample of incorrect words. This speeds up training considerably compared to computing the softmax over all logits and then estimating the cross-entropy loss. After training, the model can be used normally, using the regular softmax function to compute all the class probabilities based on all the logits.

First we need to build a function that generates strings based on a grammar. The grammar will be represented as a list of possible transitions for each state. A transition specifies the string to output (or a grammar to generate it) and the next state.

In [141]:
default_reber_grammar = [
    [("B", 1)],           # (state 0) =B=>(state 1)
    [("T", 2), ("P", 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
    [("S", 2), ("X", 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
    [("T", 3), ("V", 5)], # and so on...
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    [("E", None)]]        # (state 6) =E=>(terminal state)

embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    [("E", None)]]

def generate_string(grammar):
    state = 0
    output = []
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

Let's generate a few strings based on the default Reber grammar:

In [142]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(default_reber_grammar), end=" ")

Looks good. Now let's generate a few strings based on the embedded Reber grammar:

In [143]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(embedded_reber_grammar), end=" ")

Okay, now we need a function to generate strings that do not respect the grammar. We could generate a random string, but the task would be a bit too easy, so instead we will generate a string that respects the grammar, and we will corrupt it by changing just one character:

In [144]:
POSSIBLE_CHARS = "BEPSTVX"

def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    return good_string[:index] + bad_char + good_string[index + 1:]

Let's look at a few corrupted strings:

In [145]:
np.random.seed(42)

for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=" ")

We cannot feed strings directly to an RNN, so we need to encode them somehow. One option would be to one-hot encode each character. Another option is to use embeddings. Let's go for the second option (but since there are just a handful of characters, one-hot encoding would probably be a good option as well). For embeddings to work, we need to convert each string into a sequence of character IDs. Let's write a function for that, using each character's index in the string of possible characters "BEPSTVX":

In [146]:
def string_to_ids(s, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in s]

In [147]:
string_to_ids("BTTTXXVVETE")

We can now generate the dataset, with 50% good strings, and 50% bad strings:

In [148]:
def generate_dataset(size):
    good_strings = [
        string_to_ids(generate_string(embedded_reber_grammar))
        for _ in range(size // 2)
    ]
    bad_strings = [
        string_to_ids(generate_corrupted_string(embedded_reber_grammar))
        for _ in range(size - size // 2)
    ]
    all_strings = good_strings + bad_strings
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array([[1.] for _ in range(len(good_strings))] +
                 [[0.] for _ in range(len(bad_strings))])
    return X, y

In [149]:
np.random.seed(42)

X_train, y_train = generate_dataset(10000)
X_valid, y_valid = generate_dataset(2000)

Let's take a look at the first training sequence:

In [150]:
X_train[0]

What class does it belong to?

In [151]:
y_train[0]

Perfect! We are ready to create the RNN to identify good strings. We build a simple sequence binary classifier:

In [152]:
np.random.seed(42)
tf.random.set_seed(42)

embedding_size = 5

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    tf.keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS),
                              output_dim=embedding_size),
    tf.keras.layers.GRU(30),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
optimizer = tf.keras.optimizers.SGD(learning_rate=0.02, momentum = 0.95,
                                    nesterov=True)
model.compile(loss="binary_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Now let's test our RNN on two tricky strings: the first one is bad while the second one is good. They only differ by the second to last character. If the RNN gets this right, it shows that it managed to notice the pattern that the second letter should always be equal to the second to last letter. That requires a fairly long short-term memory (which is the reason why we used a GRU cell).

In [153]:
test_strings = ["BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE",
                "BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE"]
X_test = tf.ragged.constant([string_to_ids(s) for s in test_strings], ragged_rank=1)

y_proba = model.predict(X_test)
print()
print("Estimated probability that these are Reber strings:")
for index, string in enumerate(test_strings):
    print("{}: {:.2f}%".format(string, 100 * y_proba[index][0]))

Ta-da! It worked fine. The RNN found the correct answers with very high confidence. :)

**Work in progress**

If you would like to contribute a solution for an exercise, please submit a Pull Request, that would be greatly appreciated. Please aim for simple & flat code with as little boilerplate as possible: optimize for readability rather than efficiency.